# SimpleStrategy 백테스트

이동평균선 기반 추세 추종 전략 백테스트

In [1]:
from src.upbit.upbit_api import UpbitCandleInterval
%env ENV_PROFILE=local
from datetime import datetime, timedelta

# 필요한 라이브러리 import
import pandas as pd

from src.backtest.backtest_builder import BacktestBuilder
from src.backtest.commission_config import CommissionConfig
from src.backtest.data_feed.pandas import PandasDataFeedConfig
from src.backtest.strategy.simple_strategy import SimpleStrategy
# 데이터베이스 관련
from src.container import ApplicationContainer

di_container = ApplicationContainer()

print("✅ 라이브러리 import 완료")

env: ENV_PROFILE=local
✅ 라이브러리 import 완료


In [2]:
upbit_api = di_container.upbit_api()
result = upbit_api.get_candles(interval=UpbitCandleInterval.MINUTE_1, count=10)
result

,open,high,low,close,volume,value,timestamp
index,,,,,,,
2025-12-10 08:14:00+09:00,137147000.0,137254000.0,137000000.0,137000000.0,2.175795,2.982966e+08,2025-12-09 23:14:00+00:00
2025-12-10 08:13:00+09:00,137219000.0,137307000.0,137147000.0,137264000.0,1.468633,2.015768e+08,2025-12-09 23:13:00+00:00
2025-12-10 08:12:00+09:00,137185000.0,137297000.0,137185000.0,137219000.0,2.844091,3.904259e+08,2025-12-09 23:12:00+00:00
2025-12-10 08:11:00+09:00,137117000.0,137186000.0,137116000.0,137148000.0,0.887335,1.216776e+08,2025-12-09 23:11:00+00:00
2025-12-10 08:10:00+09:00,137115000.0,137117000.0,137104000.0,137116000.0,0.212256,2.910376e+07,2025-12-09 23:10:00+00:00
2025-12-10 08:09:00+09:00,137114000.0,137117000.0,137114000.0,137115000.0,0.402399,5.517518e+07,2025-12-09 23:09:00+00:00
2025-12-10 08:08:00+09:00,137117000.0,137117000.0,137068000.0,137114000.0,0.181373,2.486812e+07,2025-12-09 23:08:00+00:00
2025-12-10 08:07:00+09:00,137150000.0,137176000.0,137068000.0,137117000.0,3.003335,4.118731e+08,2025-12-09 23:07:00+00:00
2025-12-10 08:06:00+09:00,137054000.0,137150000.0,137022000.0,137132000.0,0.759809,1.041536e+08,2025-12-09 23:06:00+00:00


In [ ]:
# 데이터베이스에서 BTC-KRW 1분봉 데이터 가져오기

repo = di_container.candle_minute1_repository()

# 최근 7일 데이터 조회 (1분봉 약 10,000개)
end_date = datetime.now() - timedelta(days=30)
start_date = end_date - timedelta(days=1)

candles = repo.get_candles(
    ticker="KRW-BTC",
    start_datetime=start_date,
    end_datetime=end_date
)

df = pd.DataFrame([{
    'datetime': c.timestamp,
    'open': c.open,
    'high': c.high,
    'low': c.low,
    'close': c.close,
    'volume': c.volume
} for c in candles])

# 인덱스를 DatetimeIndex로 설정 (backtrader 요구사항)
df.set_index('datetime', inplace=True)
df.sort_index(inplace=True)  # 시간 순 정렬

print(f"✅ 데이터 로드 완료: {len(df)}개 캔들")
print(f"📅 기간: {df.index[0]} ~ {df.index[-1]}")
print(f"\n데이터 샘플:")
print(df.tail())

In [ ]:
# 백테스트 설정 및 실행
print("🚀 백테스트 시작...\n")

# 데이터 피드 설정
data_config = PandasDataFeedConfig.create(df, name='BTC-KRW-1m')

# 백테스트 실행 (1분봉용 파라미터)
results = (
    BacktestBuilder()
    .with_initial_cash(1_000_000)  # 초기 자본 100만원
    .with_commission(CommissionConfig.stock(0.0005))  # Upbit 수수료 0.05%
    .with_strategy(SimpleStrategy, ma_period=20)  # 20분 이동평균 전략 (1분봉용)
    .add_data(data_config)
    .run_and_plot()  # 실행 + 차트 출력
)

print("\n✅ 백테스트 완료!")

In [ ]:
# 백테스트 결과 분석
strategy = results[0]

# Cerebro에서 최종 포트폴리오 가치 가져오기
final_value = strategy.broker.getvalue()
initial_cash = 1_000_000

# 수익률 계산
total_return = ((final_value - initial_cash) / initial_cash) * 100

print("=" * 50)
print("📈 백테스트 결과 요약")
print("=" * 50)
print(f"초기 자본: {initial_cash:,.0f}원")
print(f"최종 자산: {final_value:,.2f}원")
print(f"총 수익률: {total_return:+.2f}%")
print(f"손익: {final_value - initial_cash:+,.2f}원")
print("=" * 50)

## 💡 사용 팁

### 파라미터 조정 (1분봉용)
```python
# 이동평균 기간 변경
.with_strategy(SimpleStrategy, ma_period=5)   # 5분 단기 전략
.with_strategy(SimpleStrategy, ma_period=20)  # 20분 중기 전략 (기본)
.with_strategy(SimpleStrategy, ma_period=60)  # 60분(1시간) 장기 전략

# 초기 자본 변경
.with_initial_cash(5_000_000)  # 500만원

# 수수료 변경
.with_commission(CommissionConfig.stock(0.001))  # 0.1%
```

### 데이터 기간 조정
```python
# 최근 3일
start_date = end_date - timedelta(days=3)

# 최근 14일
start_date = end_date - timedelta(days=14)

# 특정 기간
start_date = datetime(2024, 1, 1)
end_date = datetime(2024, 1, 31)
```

### 다른 종목 테스트
```python
# 이더리움
candles = repo.get_candles(ticker="KRW-ETH", start_datetime=start_date, end_datetime=end_date)

# 리플
candles = repo.get_candles(ticker="KRW-XRP", start_datetime=start_date, end_datetime=end_date)
```

### ⚠️ 주의사항
- 1분봉 데이터는 양이 많으므로 기간을 7~14일 이내로 제한하는 것을 권장합니다
- 이동평균 기간은 5~60분 사이가 적절합니다 (일봉과 달리 짧은 기간 사용)
- 데이터베이스에 데이터가 없으면 빈 결과가 나올 수 있습니다